In [2]:
from pathlib import Path
import numpy as np

import napari 

from ngio import open_ome_zarr_plate

In [ ]:
# Set Parameters

zarr_fld = Path("/Volumes/active/rhornb/fractal_paper/data/AssayPlate_Greiner_CELLSTAR655090.zarr")
ome_plate = open_ome_zarr_plate(zarr_fld)

row = "B"
column = "02"
label = 4

ref_cycle = 1
path = "0"

In [ ]:
# Load labelled nuclei and cells

well = f"{row}/{column}"
ome_zarr = ome_plate.get_image(row=row, column=column, image_path=str(ref_cycle))

roi_table = ome_zarr.get_table("embryo_ROI_table")
roi = [r for r in roi_table.rois() if r.name == str(label)][0]

lbls = ome_zarr.get_masked_label("nuclei", "embryo", "embryo_ROI_table", path=path)
lbl = lbls.get_roi(label)

lbls_cells = ome_zarr.get_masked_label("cells", "embryo", "embryo_ROI_table", path=path)
lbl_cells = lbls_cells.get_roi(label)

pixel_size = lbls.pixel_size.zyx

In [ ]:
# Split labels into two halves along XY plane, propagated along Z for visualization

def cut_labels_xy_half_propagate_z(label1, label2, start_angle=0.0):
    """
    Split 3D label volumes (Z, H, W) into 2 half-circle XY regions, 
    propagated identically along Z.
    
    Parameters:
        label1, label2 : np.ndarray
            3D label volumes of shape (Z, H, W)
        start_angle : float
            Rotation offset in radians (default 0.0)
    
    Returns:
        out1, out2 : np.ndarray
            The two output volumes corresponding to each half
    """
    assert label1.shape == label2.shape, "Label shapes must match"
    Z, H, W = label1.shape[:3]

    # Center coordinates
    cy, cx = (H - 1) / 2.0, (W - 1) / 2.0

    # XY coordinate grid
    y, x = np.meshgrid(np.arange(H), np.arange(W), indexing="ij")
    dx = x - cx
    dy = y - cy

    # Angle in [0, 2pi)
    angles = (np.arctan2(dy, dx) - start_angle) % (2 * np.pi)

    # Two equal halves
    edges = np.linspace(0, 2 * np.pi, 3)  # 0, pi, 2pi
    m1 = (angles >= edges[0]) & (angles < edges[1])
    m2 = (angles >= edges[1]) & (angles < edges[2])

    # Expand masks to 3D
    m1 = np.broadcast_to(m1, (Z, H, W))
    m2 = np.broadcast_to(m2, (Z, H, W))

    out1 = np.zeros_like(label1)
    out2 = np.zeros_like(label2)

    out1[m1] = label1[m1]
    out2[m2] = label2[m2]

    return out1, out2


lbl_sections, lbl_cells_sections = cut_labels_xy_half_propagate_z(
    lbl, lbl_cells)


In [ ]:
# Open in napari for visualization

viewer = napari.Viewer(ndisplay=3)
viewer.add_labels(lbl_sections,
                  scale=pixel_size,
                  blending="minimum")
viewer.add_labels(lbl_cells_sections,
                  scale=pixel_size,
                  blending="minimum")
viewer.window.set_geometry(0, 50, 900, 800)
viewer.theme = 'light'
viewer.scale_bar.visible = True
viewer.scale_bar.unit = "µm"
viewer.scale_bar.font_size  = 24
viewer.camera.zoom = 1
# viewer.screenshot("zfish_cycle12_segmentation.png")